In [ ]:
import tensorflow as tf
import os
import networks as nw

In [ ]:
# Instantiate the model
unet = nw.UNet(input_size=(256, 256, 3), num_classes=1)

out = unet.call(tf.random.normal((1, 256, 256, 3)))  # Call the model with a random input to build the model
print(out.shape) # Make sure the output shape is correct

unet.summary()

In [ ]:
def load_image(image_path):
    """Load an image, decode it, and resize to 256x256"""
    img = tf.io.read_file(image_path)               # Read image
    img = tf.image.decode_jpeg(img, channels=3)       # Decode as JPG (assuming JPG images)
    img = tf.image.resize(img, (256, 256))           # Resize to target size
    img = img / 255.0                               # Normalize to range [0, 1]
    return img

def load_mask(mask_path):
    """Load a mask, decode it, and resize to 256x256"""
    mask = tf.io.read_file(mask_path)               # Read mask image
    mask = tf.image.decode_png(mask, channels=1)     # Decode as PNG (assuming masks are single-channel)
    mask = tf.image.resize(mask, (256, 256))         # Resize to target size
    mask = mask / 255.0                             # Normalize to range [0, 1]
    return mask

def load_data(image_folder, mask_folder):
    # Get image and mask file paths
    image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
    mask_paths = [os.path.join(mask_folder, fname) for fname in os.listdir(mask_folder)]
    
    # Shuffle the paths (optional)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))

    # Load images and masks
    dataset = dataset.map(lambda x, y: (load_image(x), load_mask(y)))
    
    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=1000)  # Shuffle with buffer_size (adjust as needed)
    dataset = dataset.batch(batch_size=16)  # Batch size for training (adjust as needed)
    
    return dataset


In [ ]:
# Set the directory paths
image_folder = 'normalized_images'
mask_folder = 'normalized_masks'

# Load the dataset
train_dataset = load_data(image_folder, mask_folder)

# Check the dataset shapes (optional)
for images, masks in train_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Mask batch shape:", masks.shape)

In [ ]:
# Determine the number of GPUs available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

print(tf.__version__)


print("Is TensorFlow using GPU? ", tf.test.is_gpu_available())
print("GPU Details: ", tf.config.experimental.list_physical_devices('GPU'))


In [ ]:
# Compile the model
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
unet.fit(train_dataset, epochs=5)  # Adjust the number of epochs as needed

# Save the model
unet.save('unet_model.keras')  # Save the model in native Keras format

In [ ]:
# Set the directory paths
image_folder = 'test_normalized_images'
mask_folder = 'test_normalized_masks'

# Load the dataset
test_dataset = load_data(image_folder, mask_folder)

# Check the dataset shapes (optional)
for images, masks in test_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Mask batch shape:", masks.shape)

In [ ]:
# Load the model
loaded_model = tf.keras.models.load_model('unet_model.keras')

# Evaluate the model
loss, accuracy = loaded_model.evaluate(test_dataset)

print("Restored model, accuracy: {:5.2f}%".format(100*accuracy))

# Predict on a sample image
sample_image = load_image('normalized_images/test.png')
sample_image = tf.expand_dims(sample_image, 0)  # Add batch dimension
prediction = loaded_model.predict(sample_image)




# Jaccard Score

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

# Lists to store true masks and predicted masks
y_true_list = []
y_pred_list = []

# Iterate over the test dataset
for image, mask in test_dataset:
    preds = unet.predict(image)  # Get predicted mask
    preds = (preds > 0.5).astype(np.uint8)  # Convert to binary mask using threshold 0.5

    # Flatten the arrays for Jaccard computation
    y_true_list.append(mask.numpy().flatten())
    y_pred_list.append(preds.flatten())

# Convert lists to NumPy arrays
y_true = np.concatenate(y_true_list)
y_pred = np.concatenate(y_pred_list)


# Compute Jaccard Index (IoU)
jaccard = jaccard_score(y_true, y_pred)

print(f"Jaccard Index (IoU): {jaccard:.4f}")

# ROC Curve

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Generate predictions on the test dataset
y_true = []  # Ground truth masks (binary)
y_scores = []  # Predicted probability maps

for image, mask in test_dataset:  # Iterate over your dataset
    preds = unet.predict(image)  # Get model predictions
    y_true.append(mask.numpy().flatten())  # Flatten the ground truth
    y_scores.append(preds.flatten())  # Flatten predictions

# Convert lists to NumPy arrays
y_true = np.concatenate(y_true)
y_scores = np.concatenate(y_scores)


# Compute ROC curve and AUC
fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (AUC = {roc_auc:.4f})")
plt.plot([0, 1], [0, 1], color="gray", linestyle="--")  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()


# Compute Youden's J statistic to find the best threshold
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = _[optimal_idx]

print(f"Optimal Threshold: {optimal_threshold:.4f}")